In [17]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import pandas as pd
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

In [2]:
import numpy as np
import pandas as pd
import os
from scipy import sparse
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
import xgboost as xgb
import lightgbm as lgb
import matplotlib.pyplot as plt

data = pd.read_csv('./datasets/train_data.csv')
target = pd.read_csv('./datasets/train_target.csv', names=['label'], header=None)

data.drop(data.columns[[0]], axis=1, inplace=True)  #删除某列函数
target = target.reset_index(drop=True)

one_hot_feature=['creativeSize', 'LBS','age','carrier','consumptionAbility', 'education','gender','house','os','marriageStatus','advertiserId','campaignId', 'creativeId',
       'adCategoryId', 'productId', 'productType']
vector_feature=['interest1','interest2','interest5','kw1','kw2','topic1','topic2']



for feature in one_hot_feature:
    try:
        data[feature] = LabelEncoder().fit_transform(data[feature].apply(int))
    except:
        data[feature] = LabelEncoder().fit_transform(data[feature])

In [3]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
y = np.array(target).squeeze()
for train_index, test_index in split.split(data,  y):
    X_train = data.iloc[train_index]
    X_test = data.iloc[test_index]
    y_train = y[train_index]
    y_test = y[test_index]

In [4]:
# 处理联网类型特征
ct_train = X_train['ct'].values
ct_train = [m.split(' ') for m in ct_train]
ct_trains = []
for i in ct_train:
    index = [0, 0, 0, 0, 0]
    for j in i:
        index[int(j)] = 1
    ct_trains.append(index)

ct_test = X_test['ct'].values
ct_test = [m.split(' ') for m in ct_test]
ct_tests = []
for i in ct_test:
    index = [0, 0, 0, 0, 0]
    for j in i:
        index[int(j)] = 1
    ct_tests.append(index)

In [33]:
X_train_encoded = ct_trains
X_test_encoded = ct_tests
oc_encoder = OneHotEncoder()
for feature in one_hot_feature:
    oc_encoder.fit(data[feature].values.reshape(-1, 1))
    train_a=oc_encoder.transform(X_train[feature].values.reshape(-1, 1))
    X_train_encoded = sparse.hstack((X_train_encoded, train_a))
    test_a=oc_encoder.transform(X_test[feature].values.reshape(-1, 1))
    X_test_encoded = sparse.hstack((X_test_encoded, test_a))

In [6]:
vector_feature=['interest1','interest2','interest5','kw1','kw2','topic1','topic2']
ct_encoder = CountVectorizer(min_df=0.0009)
for feature in vector_feature:
    ct_encoder.fit(data[feature])
    train_a = ct_encoder.transform(X_train[feature])
    X_train_encoded = sparse.hstack((X_train_encoded, train_a))
    test_a = ct_encoder.transform(X_test[feature])
    X_test_encoded = sparse.hstack((X_test_encoded, test_a))

In [160]:
class Net(nn.Module):
    def __init__(self, input_dim, first_out_dim, secont_out_dim):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_dim, first_out_dim)
        self.fc2 = nn.Linear(first_out_dim, secont_out_dim)
        self.fc3 = nn.Linear(secont_out_dim, 2)
        
    def forward(self, X):
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = F.sigmoid(self.fc3(X))
        return X

In [161]:
net = Net(1176, 500, 200)

In [162]:
optimizer = optim.Adam(net.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

下面创建dataset

In [163]:
class AdvertiseDataset(Dataset):
    
    def __init__(self, data, target):
        self.data = data
        self.target = torch.from_numpy(target).type(torch.LongTensor)
    
    def __len__(self):
        return self.target.shape[0]
    
    def __getitem__(self, index):
        item = self.data.getrow(index).toarray()
        item_target = self.target[index]
        return item, item_target

In [164]:
test_data = AdvertiseDataset(X_test_encoded, y_test)
train_data = AdvertiseDataset(X_train_encoded, y_train)

In [187]:
train_dataloader = DataLoader(train_data, batch_size=10,
                        shuffle=True, num_workers=0)
test_dataloader = DataLoader(test_data, batch_size=10,
                        shuffle=True, num_workers=0)

In [189]:
net.double()
EPOCH = 3
for epoch in range(EPOCH):
    for i, data in enumerate(train_dataloader):
        inputs, labels = data
        inputs = torch.DoubleTensor(inputs)
        y_pred = net(inputs)
        loss = criterion(y_pred.squeeze(), labels)
        optimizer.zero_grad()
        print('epoch:', epoch, 'loss:', loss.item())
        loss.backward()
        optimizer.step()

epoch: 0 loss: 0.3132634616404039
epoch: 0 loss: 0.31326462293901264
epoch: 0 loss: 0.41326448477011635
epoch: 0 loss: 0.4132645890159086
epoch: 0 loss: 0.4132623983830953
epoch: 0 loss: 0.41326336842483463
epoch: 0 loss: 0.41326288497709457
epoch: 0 loss: 0.3132627594221496
epoch: 0 loss: 0.31326543427882486
epoch: 0 loss: 0.5132604180711009
epoch: 0 loss: 0.31326273974872987
epoch: 0 loss: 0.4132619046855893
epoch: 0 loss: 0.31326540137323805
epoch: 0 loss: 0.41326292662524466
epoch: 0 loss: 0.3132625541166206


KeyboardInterrupt: 